In [ ]:
#@title Install prerequisite, this will take a while
# Install prerequisite, no need to run this if you are running the notebook from
# your laptop and you already install the packages in Anaconda 

!pip install biopython
!pip install ipytree
!pip install scikit-allel
!pip install zarr

# install mafft commandline
!wget https://mafft.cbrc.jp/alignment/software/mafft_7.471-1_amd64.deb
!sudo dpkg -i mafft_7.471-1_amd64.deb
#!wget https://mafft.cbrc.jp/alignment/software/mafft-7.471-linux.tgz
#!tar -zxvf mafft-7.471-linux.tgz
#!ln -s $(pwd)/mafft-linux64/mafftdir/bin/mafft /bin/mafft

     |████████████████████████████████| 2.3MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 10.8MB 2.3MB/s 
  Created wheel for scikit-allel: filename=scikit_allel-1.3.2-cp36-cp36m-linux_x86_64.whl size=4708359 sha256=d90956a5c7b6c4f233abf3774e6036c496eebe5266e9c0909cfe1968cdea4736
  Stored in directory: /root/.cache/pip/wheels/f7/25/d0/32ddaaccfe4b1c20e701fad83533a13ff0c4b9809baa99d4bf
Successfully built scikit-allel
     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 5.8MB 6.9MB/s 
  Created wheel for asciitree: filename=asciitree-0.3.3-cp36-none-any.whl size=5037 sha256=b371ecb2d9529ff6838bfcfebfa939afacfa1ce1968869465e20b29a9f621cd5
  Stored in directory: /root/.cache/pip/wheels/1d/d9/58/9808b306744df0208fccc640d3d9952a5bc7468502d42897d5
Successfully built asciitree
--2020-11-01 00:30:33--  https://mafft.cbrc.jp/alignment/software/mafft_7.471-1_amd64.deb
Resolving mafft.cbrc.jp (maf

In [ ]:
#@title Mount Google Drive (You don't need to run this if you are running notebooks on your laptop)

from google.colab import drive

# The following command will prompt a URL for you to click and obtain the
# authorization code

drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
# Set up data folder
from pathlib import Path

# Change this to where you put your project files
DATA = Path("/content/drive/My Drive/Colab Notebooks/")

In [ ]:
# change the filename parameter to where you download the vcf.gz file
clinvar_file = "clinvar_20190909.vcf.gz"

#==========================
# Your code here
# Load the clinvar vcf file
#==========================
import gzip
import pandas as pd

def get_vcfvep_header(filename):
    with gzip.open(filename, "rb") as fi:
        for l in fi:
            l = l.decode("utf-8")
            if l.startswith("##"):
                continue
            elif l.startswith("#"):
                return l[1:].strip().split("\t")
            else:
                raise ValueError("Something wrong in the VEP file!")

vcf_header = get_vcfvep_header(DATA / clinvar_file)
clinvar = pd.read_csv(DATA / clinvar_file, comment="#", sep="\t")
# make sure the CHROM column is str instead of int
clinvar.columns = vcf_header
clinvar["CHROM"] = clinvar.CHROM.astype(str)
clinvar.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,949422,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,949427,639487,A,G,.,.,"ALLELEID=626469;CLNDISDB=MedGen:C4015293,OMIM:..."
2,1,949491,659187,G,A,.,.,"ALLELEID=626470;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,949502,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
4,1,949518,652606,C,T,.,.,"ALLELEID=626471;CLNDISDB=MedGen:C4015293,OMIM:..."


In [ ]:
pd.set_option('display.max_colwidth',1000)
clinvar["INFO"]

0          AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;ALLELEID=446939;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.10:g.949422G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001583|missense_variant;ORIGIN=1;RS=143888043
1                                                     ALLELEID=626469;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.10:g.949427A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001583|missense_variant;ORIGIN=1
2                                                     ALLELEID=626470;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcificati

## 1.1 Find all variants that have been reported to be either pathogenic or likely pathogenic (you can find this in the CLNSIG field) to non-Hodgkin lymphoma (CLNDN field). How many variants do you find? (3 pts)

In [ ]:
# Filter the variants within clinvar that are pathogenic + likely pathogenic
clinFilt = clinvar[((clinvar.INFO.str.contains("Pathogenic")) | (clinvar.INFO.str.contains("Likely_pathogenic"))) 
& (clinvar.INFO.str.contains("Non-Hodgkin_lymphoma"))]
len(clinFilt)

43

The total number of variants that are either pathogenic or likely pathogenic to non-Hodgkin lymphoma I found is 43. 

After taking into account that in clinvar, there are both pathogenic and likely-pathogenic comments that appear beyond the CLNSIG= field, I decided to search instead for just the text that appears. This is the same strategy that I applied to non-Hodgkin lymphoma. 

## 1.2. How many unique genes in total do these variants locate in (GENEINFO field)? (1 pt)

In [ ]:
# First see how many of the variants do NOT have gene info
clinFilt_gene = clinFilt[((clinFilt.INFO.str.contains("GENEINFO")))]
len(clinFilt_gene)

34

In [ ]:
# Unsure of how to do it, so I partitioned the remaining rows at GENEINFO and analyzed each row to see the unique genes that follow
filtered = clinFilt_gene.INFO.str.partition("GENEINFO")
filtered

,0,1,2
17199,"ALLELEID=362755;CLNDISDB=Human_Phenotype_Ontology:HP:0002664,MeSH:D009369,MedGen:C0027651,SNOMED_CT:108369006|Human_Phenotype_Ontology:HP:0002885,MeSH:D008527,MedGen:C0025149,OMIM:155255,Orphanet:ORPHA616|Human_Phenotype_Ontology:HP:0004808,MeSH:D015470,MedGen:C0023467,OMIM:601626,Orphanet:ORPHA519,SNOMED_CT:17788007|Human_Phenotype_Ontology:HP:0006740,MedGen:C0279680|Human_Phenotype_Ontology:HP:0006775,MeSH:D009101,MedGen:C0026764,OMIM:254500,Orphanet:ORPHA29073,SNOMED_CT:109989006,SNOMED_CT:55921005|Human_Phenotype_Ontology:HP:0012056,MeSH:D008545,MedGen:C0025202|Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|Human_Phenotype_Ontology:HP:0100834,MeSH:D015179,MedGen:C0009404,SNOMED_CT:126837005|MeSH:C562393,MedGen:C0151779,SNOMED_CT:93655004|MeSH:D009190,MedGen:C3463824,OMIM:614286,Orphanet:ORPHA52688|MedGen:C0278701;CLNDN=Neoplasm|Medulloblastoma|Acute_myeloid_leukemia|Transitional_cell_carcinoma_of_the_bladder|Multiple_myeloma|Cutaneous_melanoma...",GENEINFO,=NRAS:4893;MC=SO:0001583|missense_variant;ORIGIN=2;RS=121434596
76990,"ALLELEID=22805;CLNDISDB=Human_Phenotype_Ontology:HP:0006753,MeSH:D013274,MedGen:C0038356,OMIM:613659,SNOMED_CT:126824007|Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004;CLNDN=Neoplasm_of_stomach|Non-Hodgkin_lymphoma;CLNHGVS=NC_000002.11:g.202070652C>T;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:601762.0004;",GENEINFO,"=CASP10:843;MC=SO:0001587|nonsense,SO:0001627|intron_variant;ORIGIN=2;RS=121909775"
77000,"ALLELEID=22806;CLNDISDB=Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004;CLNDN=Non-Hodgkin_lymphoma;CLNHGVS=NC_000002.11:g.202073912_202073913insA;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Pathogenic;CLNVC=Insertion;CLNVCSO=SO:0000667;CLNVI=OMIM_Allelic_Variant:601762.0005;",GENEINFO,"=CASP10:843;MC=SO:0001589|frameshift_variant,SO:0001624|3_prime_UTR_variant;ORIGIN=2;RS=398122800"
98500,"ALLELEID=363403;CLNDISDB=Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|MeSH:C535575,MedGen:C1168401,OMIM:275355,Orphanet:ORPHA67037|MedGen:C0278701;CLNDN=Non-Hodgkin_lymphoma|Squamous_cell_carcinoma_of_the_head_and_neck|Adenocarcinoma_of_stomach;CLNHGVS=NC_000003.11:g.49412898T>A;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Likely_pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;",GENEINFO,"=RHOA:387;MC=SO:0001583|missense_variant,SO:0001623|5_prime_UTR_variant;ORIGIN=2;RS=1057519954"
98501,"ALLELEID=363401;CLNDISDB=Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|MeSH:C535575,MedGen:C1168401,OMIM:275355,Orphanet:ORPHA67037|MedGen:C0278701;CLNDN=Non-Hodgkin_lymphoma|Squamous_cell_carcinoma_of_the_head_and_neck|Adenocarcinoma_of_stomach;CLNHGVS=NC_000003.11:g.49412898T>C;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Likely_pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;",GENEINFO,"=RHOA:387;MC=SO:0001583|missense_variant,SO:0001623|5_prime_UTR_variant;ORIGIN=2;RS=1057519954"
98502,"ALLELEID=363402;CLNDISDB=Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|MeSH:C535575,MedGen:C1168401,OMIM:275355,Orphanet:ORPHA67037|MedGen:C0278701;CLNDN=Non-Hodgkin_lymphoma|Squamous_cell_carcinoma_of_the_head_and_neck|Adenocarcinoma_of_stomach;CLNHGVS=NC_000003.11:g.49412898T>G;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Likely_pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;",GENEINFO,"=RHOA:387;MC=SO:0001583|missense_variant,SO:0001623|5_prime_UTR_variant;ORIGIN=2;RS=1057519954"
98505,"ALLELEID=363400;CLNDISDB=Human_Phenotype_Ontology:HP:0011459,MedGen:C0152018,Orphanet:ORPHA70482|Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|Human_Phenotype_Ontology:HP:0100013,MeSH:D001943,MedGen:C1458155,Orphanet:ORPHA180250,SNOMED_CT:126926005

The unique genes are:

NRAS:4893

CASP10:843

RHOA:387

PIK3CA

EZH2:2146

MYC:4609

B2M:567

MAP2K1:5604

TP53:7157

There are a total of 9 unique genes that have these variants.

## 1.3 How many unique pathogenic (and likely pathogenic) non-Hodgkin lymphoma variants do you find in TCGA DLBCL mutation data? (3 pts)

In [ ]:
mutations_file = DATA / "dlbc_tcga_mutation.txt"

In [ ]:
mutations = pd.read_csv(DATA / mutations_file, sep="\t", comment="#")
mutations.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (38,50,67,80,81,82,83,84,88,89,91,98,99,111,140,157,187) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(16918, 312)

In [ ]:
mutations.head()

,gene,entrez_id,center,ncbi_build,chrom,start,end,strand,variant_classification,variant_type,ref,tumor_seq_allele1,tumor_seq_allele2,dbsnp_rs,dbsnp_val_status,tumor_sample_barcode,matched_norm_sample_barcode,match_norm_seq_allele1,match_norm_seq_allele2,tumor_validation_allele1,tumor_validation_allele2,match_norm_validation_allele1,match_norm_validation_allele2,verification_status,validation_status,mutation_status,sequencing_phase,sequence_source,validation_method,score,bam_file,sequencer,tumor_sample_uuid,matched_norm_sample_uuid,genome_change,annotation_transcript,transcript_strand,transcript_exon,transcript_position,cdna_change,...,i_dbnsfp_unisnp_ids,i_dbnsfp_uniprot_aapos,i_dbnsfp_uniprot_acc,i_dbnsfp_uniprot_id,i_dbnsfp_aaalt,i_dbnsfp_aapos,i_dbnsfp_aapos_fathmm,i_dbnsfp_aapos_sift,i_dbnsfp_aaref,i_dbnsfp_cds_strand,i_dbnsfp_codonpos,i_dbnsfp_fold-degenerate,i_dbnsfp_genename,i_dbnsfp_hg18_pos(1-coor),i_dbnsfp_phastcons100way_vertebrate,i_dbnsfp_phastcons100way_vertebrate_rankscore,i_dbnsfp_phastcons46way_placental,i_dbnsfp_phastcons46way_placental_rankscore,i_dbnsfp_phastcons46way_primate,i_dbnsfp_phastcons46way_primate_rankscore,i_dbnsfp_phylop100way_vertebrate,i_dbnsfp_phylop100way_vertebrate_rankscore,i_dbnsfp_phylop46way_placental,i_dbnsfp_phylop46way_placental_rankscore,i_dbnsfp_phylop46way_primate,i_dbnsfp_phylop46way_primate_rankscore,i_dbnsfp_refcodon,i_dbsnp_af,i_dbsnp_popfreq,i_dbsnp_strength,i_entrez_gene_id.1,i_gc_content_full,i_gencode_transcript_name,i_gencode_transcript_status,i_gencode_transcript_tags,i_gencode_transcript_type,i_gene_type,i_havana_transcript,i_refseq_mrna_id,i_secondary_variant_classification
0,ZNF343,79175,hgsc.bcm.edu,37,20,2473385,2473385,+,Frame_Shift_Del,DEL,T,T,-,NaN,NaN,TCGA-FF-8061-01A-11D-2210-10,TCGA-FF-8061-10A-01D-2210-10,T,T,.,.,.,.,Unknown,Untested,Somatic,Phase_I,WXS,none,.,NaN,Illumina HiSeq,a29278af-7ecf-403e-b6a9-623ea7879d05,08470796-8ff6-4e18-98d5-f9ee0d079517,g.chr20:2473385delT,ENST00000278772.4,-,5.0,751,c.264delA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.,.,none,NaN,0.408,ZNF343-002,KNOWN,basic|appris_principal|CCDS,protein_coding,protein_coding,OTTHUMT00000077617.1,NM_024325,NaN
1,ZNF223,7766,hgsc.bcm.edu,37,19,44570830,44570831,+,Frame_Shift_Del,DEL,AG,AG,-,rs568496344,byFrequency,TCGA-FF-8061-01A-11D-2210-10,TCGA-FF-8061-10A-01D-2210-10,AG,AG,.,.,.,.,Unknown,Untested,Somatic,Phase_I,WXS,none,.,NaN,Illumina HiSeq,a29278af-7ecf-403e-b6a9-623ea7879d05,08470796-8ff6-4e18-98d5-f9ee0d079517,g.chr19:44570830_44570831delAG,ENST00000434772.3,+,5.0,1104_1105,c.849_850delAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.,.,none,NaN,0.421,ZNF223-001,KNOWN,basic|appris_principal|CCDS,protein_coding,protein_coding,OTTHUMT00000460469.2,NaN,NaN
2,GAST,2520,hgsc.bcm.edu,37,17,39872062,39872064,+,In_Frame_Del,DEL,GAA,GAA,-,NaN,NaN,TCGA-FF-8061-01A-11D-2210-10,TCGA-FF-8061-10A-01D-2210-10,GAA,GAA,.,.,.,.,Unknown,Untested,Somatic,Phase_I,WXS,none,.,NaN,Illumina HiSeq,a29278af-7ecf-403e-b6a9-623ea7879d05,08470796-8ff6-4e18-98d5-f9ee0d079517,g.chr17:39872062_39872064delGAA,ENST00000329402.3,+,3.0,311_313,c.244_246delGAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.,.,none,NaN,0.562,GAST-001,KNOWN,basic|appris_principal|CCDS,protein_coding,protein_coding,OTTHUMT00000257409.1,NaN,NaN
3,ATAD5,79915,hgsc.bcm.edu,37,17,29221631,29221635,+,Frame_Shift_Del,DEL,TCTCT,TCTCT,-,NaN,NaN,TCGA-FF-8061-01A-11D-2210-10,TCGA-FF-8061-10A-01D-2210-10,TCTCT,TCTCT,.,.,.,.,Unknown,Untested,Somatic,Phase_I,WXS,none,.,NaN,Illumina HiSeq,a29278af-7ecf-403e-b6a9-623ea7879d05,08470796-8ff6-4e18-98d5-f9ee0d079517,g.chr17:29221631_29221635delTCTCT,ENST00000321990.4,+,22.0,5725_5729,c.5347_5351delTCTCT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
clinFilt.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
11448,1,46738430,6192,T,A,.,.,"ALLELEID=21231;CLNDISDB=Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004;CLNDN=Non-Hodgkin_lymphoma;CLNHGVS=NC_000001.10:g.46738430T>A;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:603615.0002|UniProtKB_(protein):Q92698"
17199,1,115258744,375876,C,A,.,.,"ALLELEID=362755;CLNDISDB=Human_Phenotype_Ontology:HP:0002664,MeSH:D009369,MedGen:C0027651,SNOMED_CT:108369006|Human_Phenotype_Ontology:HP:0002885,MeSH:D008527,MedGen:C0025149,OMIM:155255,Orphanet:ORPHA616|Human_Phenotype_Ontology:HP:0004808,MeSH:D015470,MedGen:C0023467,OMIM:601626,Orphanet:ORPHA519,SNOMED_CT:17788007|Human_Phenotype_Ontology:HP:0006740,MedGen:C0279680|Human_Phenotype_Ontology:HP:0006775,MeSH:D009101,MedGen:C0026764,OMIM:254500,Orphanet:ORPHA29073,SNOMED_CT:109989006,SNOMED_CT:55921005|Human_Phenotype_Ontology:HP:0012056,MeSH:D008545,MedGen:C0025202|Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|Human_Phenotype_Ontology:HP:0100834,MeSH:D015179,MedGen:C0009404,SNOMED_CT:126837005|MeSH:C562393,MedGen:C0151779,SNOMED_CT:93655004|MeSH:D009190,MedGen:C3463824,OMIM:614286,Orphanet:ORPHA52688|MedGen:C0278701;CLNDN=Neoplasm|Medulloblastoma|Acute_myeloid_leukemia|Transitional_cell_carcinoma_of_the_bladder|Multiple_myeloma|Cutaneous_melanoma..."
17201,1,115258744,13901,C,T,.,.,"AF_EXAC=0.00001;ALLELEID=28940;CLNDISDB=Human_Phenotype_Ontology:HP:0002885,MeSH:D008527,MedGen:C0025149,OMIM:155255,Orphanet:ORPHA616|Human_Phenotype_Ontology:HP:0004808,MeSH:D015470,MedGen:C0023467,OMIM:601626,Orphanet:ORPHA519,SNOMED_CT:17788007|Human_Phenotype_Ontology:HP:0006740,MedGen:C0279680|Human_Phenotype_Ontology:HP:0006775,MeSH:D009101,MedGen:C0026764,OMIM:254500,Orphanet:ORPHA29073,SNOMED_CT:109989006,SNOMED_CT:55921005|Human_Phenotype_Ontology:HP:0012056,MeSH:D008545,MedGen:C0025202|Human_Phenotype_Ontology:HP:0012209,MedGen:C0349639,OMIM:607785,Orphanet:ORPHA86834|Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|Human_Phenotype_Ontology:HP:0100834,MeSH:D015179,MedGen:C0009404,SNOMED_CT:126837005|MeSH:C562393,MedGen:C0151779,SNOMED_CT:93655004|MeSH:D009190,MedGen:C3463824,OMIM:614286,Orphanet:ORPHA52688|MedGen:C0278701|MedGen:C2674723,OMIM:614470,Orphanet:ORPHA268114|MedGen:C2750732,OMIM:613224|MedGen:CN517202;CLNDN=Medulloblastoma|Acu..."
17203,1,115258745,13899,C,G,.,.,"ALLELEID=28938;CLNDISDB=Human_Phenotype_Ontology:HP:0002885,MeSH:D008527,MedGen:C0025149,OMIM:155255,Orphanet:ORPHA616|Human_Phenotype_Ontology:HP:0004808,MeSH:D015470,MedGen:C0023467,OMIM:601626,Orphanet:ORPHA519,SNOMED_CT:17788007|Human_Phenotype_Ontology:HP:0005600,MedGen:C1842036,OMIM:137550|Human_Phenotype_Ontology:HP:0006740,MedGen:C0279680|Human_Phenotype_Ontology:HP:0006775,MeSH:D009101,MedGen:C0026764,OMIM:254500,Orphanet:ORPHA29073,SNOMED_CT:109989006,SNOMED_CT:55921005|Human_Phenotype_Ontology:HP:0012056,MeSH:D008545,MedGen:C0025202|Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004|Human_Phenotype_Ontology:HP:0100834,MeSH:D015179,MedGen:C0009404,SNOMED_CT:126837005|MeSH:C562393,MedGen:C0151779,SNOMED_CT:93655004|MeSH:D009190,MedGen:C3463824,OMIM:614286,Orphanet:ORPHA52688|MedGen:C0278701|MedGen:C0699790,OMIM:114500,SNOMED_CT:269533000;CLNDN=Medulloblastoma|Acute_myeloid_leukemia|Congenital_giant_melanocytic_nevus|Transitional_cell_carcinom..."
76990,2,202070652,7766,C,T,.,.,"ALLELEID=22805;CLNDISDB=Human_Phenotype_Ontology:HP:0006753,MeSH:D013274,MedGen:C0038356,OMIM:613659,SNOMED_CT:126824007|Human_Phenotype_Ontology:HP:0012539,MedGen:C4721532,OMIM:605027,SNOMED_CT:1929004;CLNDN=Neoplasm_of_stomach|Non-Hodgkin_lymphoma;CLNHGVS=NC_000002.11:g.202070652C>T;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Pathogenic;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;CLNVI=OMIM_Allelic_Variant:601762.0004;GENEINFO=CASP

In [ ]:
merged = pd.merge(clinFilt_gene, mutations, how='inner', left_on=["CHROM", "POS", "ALT"], right_on=["chrom", "start", "tumor_seq_allele2"])
merged.shape

(7, 320)

In order to check how many of the variants from the previous filtered clinvar appear in TCGA DLBCL mutation data, I cross-checked chromosome number, the alternate allele, and the positions of the variants. 

The final number of variants that appear in the mutation data is 7.

## 1.4. Categorize the 48 DLBCL samples into three tiers as described above. How many samples do you have in each category? (1 pts)

In [ ]:
mutations["tumor_sample_barcode"].nunique()

48

In [ ]:
mutations.shape

(16918, 312)

In [ ]:
Tier_1 = merged.groupby("tumor_sample_barcode", as_index=False).count()
Tier_1["tumor_sample_barcode"]

0    TCGA-FA-8693-01A-11D-2397-10
1    TCGA-FA-A7Q1-01A-11D-A382-10
2    TCGA-FM-8000-01A-11D-2210-10
3    TCGA-G8-6907-01A-11D-2210-10
4    TCGA-GR-7353-01A-11D-2210-10
5    TCGA-GS-A9TY-01A-11D-A38X-10
Name: tumor_sample_barcode, dtype: object

In [ ]:
# Thus Tier 1 has 6 samples in it that are from unique DLBCL samples with exact pathogenic variants.

In [ ]:
# Filter out the pathogenic + likely-pathogenic samples for the rest - Tier2
clinRest = clinvar[~clinvar.isin(clinFilt)].dropna()
clinRest.shape

(474797, 8)

In [ ]:
clinRest = mutations[~mutations["tumor_sample_barcode"].isin(Tier_1["tumor_sample_barcode"])]
clinRest.shape

(14753, 312)

In [ ]:
unique_genes = ["NRAS", "CASP10", "RHOA", "PIK3CA", "EZH2", "MYC", "B2M", "MAP2K1", "TP53"]

In [ ]:
Tier_2_genes = clinRest[clinRest["gene"].isin(unique_genes)]
Tier_2 = Tier_2_genes.groupby("tumor_sample_barcode", as_index=False).count()
Tier_2["tumor_sample_barcode"]

0     TCGA-FA-A4XK-01A-11D-A31X-10
1     TCGA-FA-A6HN-01A-11D-A31X-10
2     TCGA-FA-A82F-01A-11D-A382-10
3     TCGA-FF-8041-01A-11D-2210-10
4     TCGA-FF-8043-01A-11D-2210-10
5     TCGA-FF-8046-01A-11D-2210-10
6     TCGA-FF-8047-01A-11D-2210-10
7     TCGA-FF-8061-01A-11D-2210-10
8     TCGA-FF-8062-01A-11D-2210-10
9     TCGA-G8-6324-01A-11D-2210-10
10    TCGA-G8-6325-01A-11D-2210-10
11    TCGA-G8-6914-01A-11D-2210-10
12    TCGA-GR-7351-01A-11D-2210-10
13    TCGA-GR-A4D4-01A-11D-A31X-10
14    TCGA-GR-A4D9-01B-11D-A31X-10
15    TCGA-GS-A9TV-01A-11D-A382-10
16    TCGA-GS-A9TW-01A-11D-A382-10
17    TCGA-RQ-A6JB-01A-11D-A31X-10
18    TCGA-RQ-AAAT-01A-11D-A38X-10
Name: tumor_sample_barcode, dtype: object

In [ ]:
# As Tier_2 has 19 rows, then Tier 2 consists of 19 DLBCL samples.

In [ ]:
Tier_3_samples = mutations[((~mutations["tumor_sample_barcode"].isin(Tier_1["tumor_sample_barcode"])) 
                  & (~mutations["tumor_sample_barcode"].isin(Tier_2["tumor_sample_barcode"])))]

Tier_3 = Tier_3_samples.groupby("tumor_sample_barcode", as_index=False).count()
Tier_3["tumor_sample_barcode"]

0     TCGA-FA-A4BB-01A-11D-A31X-10
1     TCGA-FA-A6HO-01A-11D-A31X-10
2     TCGA-FA-A7DS-01A-11D-A382-10
3     TCGA-FA-A86F-01A-11D-A382-10
4     TCGA-FF-8042-01A-11D-2210-10
5     TCGA-FF-A7CQ-01A-11D-A382-10
6     TCGA-FF-A7CR-01A-11D-A382-10
7     TCGA-FF-A7CW-01A-11D-A382-10
8     TCGA-FF-A7CX-01A-12D-A382-10
9     TCGA-G8-6326-01A-11D-2210-10
10    TCGA-G8-6906-01A-11D-2210-10
11    TCGA-G8-6909-01A-11D-2210-10
12    TCGA-GR-A4D5-01A-11D-A31X-10
13    TCGA-GR-A4D6-01A-11D-A31X-10
14    TCGA-GS-A9TQ-01A-11D-A382-10
15    TCGA-GS-A9TT-01A-11D-A382-10
16    TCGA-GS-A9TU-01A-11D-A382-10
17    TCGA-GS-A9TX-01A-11D-A382-10
18    TCGA-GS-A9TZ-01A-11D-A38X-10
19    TCGA-GS-A9U3-01A-11D-A38X-10
20    TCGA-GS-A9U4-01A-11D-A38X-10
21    TCGA-RQ-A68N-01A-11D-A31X-10
22    TCGA-VB-A8QN-01A-11D-A382-10
Name: tumor_sample_barcode, dtype: object

In [ ]:
# Tier 3 contains the rest of the samples, which are 23 and are the barcodes listed above.

Based on the above, we can see that:

Tier 1 has 6 samples

Tier 2 has 19 samples

Tier 3 has 23 samples.